In [1]:
import tvm
from tvm import te
import numpy as np

In [4]:
a = np.arange(12, dtype='float32').reshape((3, 4))
print(a)
print(np.tril(a))

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[[ 0.  0.  0.  0.]
 [ 4.  5.  0.  0.]
 [ 8.  9. 10.  0.]]


In [6]:
n, m = te.var('n'), te.var('m')
A = te.placeholder((n, m))
# if_then_else. It accepts three arguments, the first one is the condition, if true returning the second argument, otherwise returning the third one
B = te.compute(A.shape, lambda i, j: te.if_then_else(i >= j, A[i,j], 0.0))

In [7]:
b = tvm.nd.array(np.empty_like(a))
s = te.create_schedule(B.op)
print(tvm.lower(s, [A, B], simple_mode=True))
mod = tvm.build(s, [A, B])
mod(tvm.nd.array(a), b)
b

@main = primfn(placeholder_1: handle, compute_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {placeholder: Buffer(placeholder_2: Pointer(float32), float32, [(stride: int32*n: int32)], [], type="auto"),
             compute: Buffer(compute_2: Pointer(float32), float32, [(stride_1: int32*n)], [], type="auto")}
  buffer_map = {placeholder_1: placeholder, compute_1: compute}
  preflattened_buffer_map = {placeholder_1: placeholder_3: Buffer(placeholder_2, float32, [n, m: int32], [stride, stride_2: int32], type="auto"), compute_1: compute_3: Buffer(compute_2, float32, [n, m], [stride_1, stride_3: int32], type="auto")} {
  for (i: int32, 0, n) {
    for (j: int32, 0, m) {
      compute[((i*stride_1) + (j*stride_3))] = @tir.if_then_else((j <= i), placeholder[((i*stride) + (j*stride_2))], 0f32, dtype=float32)
    }
  }
}




<tvm.nd.NDArray shape=(3, 4), cpu(0)>
array([[ 0.,  0.,  0.,  0.],
       [ 4.,  5.,  0.,  0.],
       [ 8.,  9., 10.,  0.]], dtype=float32)